In [291]:

#    Multi-Disciplinary Design Optimisation of a Hydrogen Powered Commercial Aircraft (MDOHPCA)
#    this is the master file that runs each of the individual disciplines 
#    It sets up each of the analyses, then converts it into an openMDAO problem with a certain structure, then solves for minimum mass



In [292]:

import pandas as pd
import numpy as np
import openmdao.api as om

In [293]:
class MDOHPCA(om.Group):
    """
    Top level group containing the MDO.
    """           
    def setup(self):
        """
        set up each person's disciplines that can contain analyses
        """
        self.add_subsystem('aero', aero())
        self.add_subsystem('sys', sys())
        self.add_subsystem('stab', stab())
        self.add_subsystem('struct', struct())
        
        self.add_subsystem('mass', mass())
        
    def configure(self):
        #promote all variables (lazy option, they can be connected individually)
        self.promotes('sys',any=['*'])
        self.promotes('struct',any=['*'])
        
        self.promotes('mass',any=['*'])

In [294]:
class mass(om.ExplicitComponent):
    def setup(self):
        self.add_input('systems_mass')
        self.add_input('weight_structures')
        self.add_input('cog_structures', shape = (2,))
        self.add_input('systems_CG')
        
        self.add_output('m')
        prob.model.add_objective('m')
        
        self.add_output('total_CG', shape = (1,2))
        
    def setup_partials(self):
        self.declare_partials('*', '*', method = 'fd')
        
    def compute(self,inputs,outputs):
        #calculation of mass from systems and structures
        mass = inputs['systems_mass'] + inputs['weight_structures']
        
        outputs['m'] = mass
        
        #calculation of CG from systems and structures
        struct_CG = inputs['cog_structures']
        sys_CG = inputs['systems_CG']
        
        struct_comp_x = struct_CG[0]*inputs['weight_structures']
        struct_comp_y = struct_CG[1]*inputs['weight_structures']
        
        sys_comp_x = sys_CG*inputs['systems_mass']
        
        total_CG_x = (struct_comp_x + sys_comp_x)/mass
        total_CG_y = struct_comp_y/mass
        
        total_CG = [total_CG_x, total_CG_y]
        outputs['total_CG'] = total_CG

In [295]:
%run struct/structsetup.ipynb
#%run stab/stabsetup.ipynb
%run systems/systemssetup.ipynb
class aero(om.ExplicitComponent):
    pass
class stab(om.ExplicitComponent):
    pass

In [296]:
prob = om.Problem(model = MDOHPCA())
#prob.model.add_subsystem('aero', om.Group())

In [297]:
#setting inputs to reduce ambiguities where inputs differ in different disciplines
prob.model.set_input_defaults('CL', val = 0.6)
prob.model.set_input_defaults('sweep', val = 15)
prob.model.set_input_defaults('c', val = 4)
prob.model.set_input_defaults('b', val = 60)
prob.model.set_input_defaults('root_x', val = 10)
prob.model.set_input_defaults('m', val = 240000)


prob.model.add_design_var('tank_ratio', lower = 0.3, upper = 1)
prob.model.set_input_defaults('tank_ratio', val = 1)


In [298]:
#declare problem options (driver, optimiser)

#prob.nonlinear_solver = om.NonlinearBlockGS()
#prob.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.nonlinear_solver.options['iprint'] = 2
prob.model.nonlinear_solver.options['maxiter'] = 200

#prob.nonlinear_solver.options['maxiter'] = 100

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-5

#setup problem to be run
prob.setup()

In [299]:
#%run aero/aerosetup.ipynb

In [300]:
prob.run_model()

NL: NLBGS 1 ; 920802135 1
NL: NLBGS 2 ; 4503807.84 0.00489117875
NL: NLBGS 3 ; 141272.013 0.000153422769
NL: NLBGS 4 ; 16031.3286 1.74101775e-05
NL: NLBGS 5 ; 1750.01083 1.90052864e-06
NL: NLBGS 6 ; 190.242384 2.06605063e-07
NL: NLBGS 7 ; 20.6717877 2.24497608e-08
NL: NLBGS 8 ; 2.24609191 2.43927748e-09
NL: NLBGS 9 ; 0.244047691 2.65038146e-10
NL: NLBGS 10 ; 0.0265168322 2.87975356e-11
NL: NLBGS Converged


In [301]:
#prob.run_driver()

In [302]:
prob.model.get_design_vars()

{'tank_ratio': {'scaler': None,
  'adder': None,
  'name': 'tank_ratio',
  'upper': 1.0,
  'lower': 0.3,
  'ref': None,
  'ref0': None,
  'units': None,
  'cache_linear_solution': False,
  'indices': None,
  'flat_indices': False,
  'parallel_deriv_color': None,
  'source': '_auto_ivc.v2',
  'orig': (None, 'tank_ratio'),
  'distributed': False,
  'total_adder': None,
  'total_scaler': None,
  'size': 1,
  'global_size': 1}}

In [303]:
prob.model.get_val('tank_ratio')

array([1.])

In [304]:
#get properties of specific constants in disciplines
#prob.model.sys.options._dict['pp_mass']

prob.model.list_outputs()

32 Explicit Output(s) in 'model'

varname                         val                 
------------------------------  --------------------
sys
  fuel_mass_calc
    fuel_mass                   [59580.6163499]     
  tanks
    tank1_mass_full             [43895.21152577]    
    tank1_mass_empty            [8146.84171583]     
    tank2_mass_full             [29613.05740932]    
    tank2_mass_empty            [5780.81086935]     
    CGtp                        [51.08337761]       
    tank1_x                     [1.]                
    tank2_x                     [1.]                
    length_fuselage             [82.91697892]       
  pipes
    pipe_CG                     [10.84355971]       
    pipe_mass                   [60.74845366]       
  engine
    engine_mass                 [1639.31494409]     
    engine_x                    [14.82308546]       
    engine_y                    [18.]               
  actuator
    each_flap_actuator_mass     [2.55]              
    each

[('sys.fuel_mass_calc.fuel_mass', {'val': array([59580.6163499])}),
 ('sys.tanks.tank1_mass_full', {'val': array([43895.21152577])}),
 ('sys.tanks.tank1_mass_empty', {'val': array([8146.84171583])}),
 ('sys.tanks.tank2_mass_full', {'val': array([29613.05740932])}),
 ('sys.tanks.tank2_mass_empty', {'val': array([5780.81086935])}),
 ('sys.tanks.CGtp', {'val': array([51.08337761])}),
 ('sys.tanks.tank1_x', {'val': array([1.])}),
 ('sys.tanks.tank2_x', {'val': array([1.])}),
 ('sys.tanks.length_fuselage', {'val': array([82.91697892])}),
 ('sys.pipes.pipe_CG', {'val': array([10.84355971])}),
 ('sys.pipes.pipe_mass', {'val': array([60.74845366])}),
 ('sys.engine.engine_mass', {'val': array([1639.31494409])}),
 ('sys.engine.engine_x', {'val': array([14.82308546])}),
 ('sys.engine.engine_y', {'val': array([18.])}),
 ('sys.actuator.each_flap_actuator_mass', {'val': array([2.55])}),
 ('sys.actuator.each_aileron_actuator_mass', {'val': array([0.1275])}),
 ('sys.actuator.actuators_CG_x', {'val': a

In [305]:
arr = [4,2,3,4,5,6]
np.argmin(arr)

1

In [306]:
arr[np.argmin(arr)]

2